In [1]:
!pip install kloppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 861.7 kB/s eta 0:00:00


In [2]:
from kloppy import datasets, wyscout
import json
import pandas as pd
import numpy as np

In [3]:
manu_westham = datasets.load('wyscout', match_id = 2499724)

In [4]:
teams_json = '../input/football-event-data/teams.json'
with open(teams_json) as f:
    data = json.load(f)
    teams = pd.json_normalize(data, sep = "_")
teams_map = dict()
for i, team in teams.iterrows():
    teams_map[team['wyId']] = team['name']
teams_map

{1613: 'Newcastle United',
 692: 'Celta de Vigo',
 691: 'Espanyol',
 696: 'Deportivo Alav\\u00e9s',
 695: 'Levante',
 3795: 'Troyes',
 698: 'Getafe',
 2454: "Borussia M'gladbach",
 1673: 'Huddersfield Town',
 678: 'Athletic Club',
 679: 'Atl\\u00e9tico Madrid',
 3766: 'Olympique Lyonnais',
 3767: 'PSG',
 674: 'Valencia',
 675: 'Real Madrid',
 676: 'Barcelona',
 714: 'Las Palmas',
 712: 'Legan\\u00e9s',
 3204: 'SPAL',
 10531: 'Swansea City',
 3771: 'Olympique Marseille',
 3770: 'Nantes',
 3775: 'Nice',
 3774: 'Rennes',
 3779: 'Strasbourg',
 701: 'Eibar',
 1659: 'AFC Bournemouth',
 1651: 'Brighton & Hove Albion',
 2443: 'Werder Bremen',
 2446: 'Bayer Leverkusen',
 2447: 'Borussia Dortmund',
 2444: 'Bayern M\\u00fcnchen',
 2445: 'Stuttgart',
 2449: 'Schalke 04',
 3157: 'Milan',
 3799: 'Angers',
 3159: 'Juventus',
 3158: 'Roma',
 3315: 'Sassuolo',
 1646: 'Burnley',
 3772: 'Bordeaux',
 2455: 'Hannover 96',
 3804: 'Dijon',
 2457: 'Hertha BSC',
 2451: 'Wolfsburg',
 2450: 'Hamburger SV',
 2453

In [5]:
json_path = '../input/wyscoutdata/matches_England.json'
with open(json_path) as f:
    data = json.load(f)
    matches = pd.json_normalize(data, sep = "_")

In [6]:
import os
os.listdir('/kaggle/input/wyscoutdata')

['matches_European_Championship.json',
 'matches_France.json',
 'matches_Italy.json',
 'matches_Spain.json',
 'matches_World_Cup.json',
 'matches_England.json',
 'matches_Germany.json']

In [7]:
matches['gameweek']

0      38
1      38
2      38
3      38
4      38
       ..
375     1
376     1
377     1
378     1
379     1
Name: gameweek, Length: 380, dtype: int64

In [8]:
matches.label.iloc[0].split(' - ')[1].split(', ')

['AFC Bournemouth', '1']

In [9]:
def split_label(string):
    home, away_home_score, away_score = string.split(' - ')
    away, home_score = away_home_score.split(', ')
    return home, away, home_score, away_score

In [10]:
matches['home_team'],matches['away_team'], matches['home_score'], matches['away_score'] = zip(*matches['label'].map(split_label))

In [11]:
scores = matches[['home_team', 'away_team', 'home_score', 'away_score', 'gameweek']]

In [12]:
teams = matches['home_team'].unique()
teams

array(['Burnley', 'Crystal Palace', 'Huddersfield Town', 'Liverpool',
       'Manchester United', 'Newcastle United', 'Southampton',
       'Swansea City', 'Tottenham Hotspur', 'West Ham United',
       'Manchester City', 'Leicester City', 'Chelsea', 'Arsenal',
       'Everton', 'AFC Bournemouth', 'Watford', 'West Bromwich Albion',
       'Stoke City', 'Brighton & Hove Albion'], dtype=object)

In [13]:
scores['home_score'] = scores['home_score'].astype(np.int)
scores['away_score'] = scores['away_score'].astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython 

In [14]:
table = {}
for team in teams:
    table[team] = {'goals': 0, 'goals_against': 0, 'points': 0, 'wins': 0, 'draws': 0, 'losses': 0}
for index, game in scores.iterrows():
    home_team = game['home_team']
    away_team = game['away_team']
    away_score = game['away_score']
    home_score = game['home_score']
    #update scores
    table[home_team]['goals'] += home_score
    table[home_team]['goals_against'] += away_score
    table[away_team]['goals'] += away_score
    table[away_team]['goals_against'] += home_score
    #update wins
    if home_score > away_score:
        table[home_team]['wins'] += 1
        table[away_team]['losses'] += 1 
    elif home_score == away_score:
        table[home_team]['draws'] += 1
        table[away_team]['draws'] += 1
    else:
        table[home_team]['losses'] += 1
        table[away_team]['wins'] += 1

In [15]:
table = pd.DataFrame().from_dict(table, orient='index')

In [16]:
table['points'] = table['wins']*3 + table['draws']

In [17]:
table.sort_values(by = 'points')

,goals,goals_against,points,wins,draws,losses
West Bromwich Albion,31,56,31,6,13,19
Stoke City,35,68,33,7,12,19
Swansea City,28,56,33,8,9,21
Southampton,37,56,36,7,15,16
Huddersfield Town,28,58,37,9,10,19
Brighton & Hove Albion,34,54,40,9,13,16
Watford,44,64,41,11,8,19
West Ham United,48,68,42,10,12,16
AFC Bournemouth,45,61,44,11,11,16
Crystal Palace,45,55,44,11,11,16


In [18]:
table.to_csv("epl_table.csv")

In [19]:
df = matches
df = df.drop(df.var()[df.var() == 0].index.values, axis=1)
for col in df.columns:
    print(col)

status
gameweek
dateutc
winner
venue
wyId
label
date
referees
duration
teamsData_1646_coachId
teamsData_1646_side
teamsData_1646_score
teamsData_1646_formation_bench
teamsData_1646_formation_lineup
teamsData_1646_formation_substitutions
teamsData_1646_scoreHT
teamsData_1659_coachId
teamsData_1659_side
teamsData_1659_score
teamsData_1659_formation_bench
teamsData_1659_formation_lineup
teamsData_1659_formation_substitutions
teamsData_1659_scoreHT
teamsData_1628_coachId
teamsData_1628_side
teamsData_1628_score
teamsData_1628_formation_bench
teamsData_1628_formation_lineup
teamsData_1628_formation_substitutions
teamsData_1628_scoreHT
teamsData_1627_coachId
teamsData_1627_side
teamsData_1627_score
teamsData_1627_formation_bench
teamsData_1627_formation_lineup
teamsData_1627_formation_substitutions
teamsData_1627_scoreHT
teamsData_1609_coachId
teamsData_1609_side
teamsData_1609_score
teamsData_1609_formation_bench
teamsData_1609_formation_lineup
teamsData_1609_formation_substitutions
teamsDa

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [20]:
week1 = matches[matches.gameweek == 1][['wyId', 'away_team', 'home_team', 'label']]
week1

,wyId,away_team,home_team,label
370,2499724,West Ham United,Manchester United,"Manchester United - West Ham United, 4 - 0"
371,2499725,Tottenham Hotspur,Newcastle United,"Newcastle United - Tottenham Hotspur, 0 - 2"
372,2499720,Manchester City,Brighton & Hove Albion,"Brighton & Hove Albion - Manchester City, 0 - 2"
373,2499721,Burnley,Chelsea,"Chelsea - Burnley, 2 - 3"
374,2499722,Huddersfield Town,Crystal Palace,"Crystal Palace - Huddersfield Town, 0 - 3"
375,2499723,Stoke City,Everton,"Everton - Stoke City, 1 - 0"
376,2499726,Swansea City,Southampton,"Southampton - Swansea City, 0 - 0"
377,2499728,AFC Bournemouth,West Bromwich Albion,"West Bromwich Albion - AFC Bournemouth, 1 - 0"
378,2499727,Liverpool,Watford,"Watford - Liverpool, 3 - 3"
379,2499719,Leicester City,Arsenal,"Arsenal - Leicester City, 4 - 3"


In [21]:
games = []
for index, game in week1.iterrows():
    df = datasets.load('wyscout', match_id = game.wyId).to_pandas()
    df['team'] = df['team_id'].apply(lambda x: teams_map[int(x)])
    games.append(df)

In [22]:
gameweek1 = pd.concat(games)

In [23]:
eventdf = gameweek1.groupby(['team', 'event_type']).size().reset_index()
eventdf[eventdf.event_type == 'PASS'].sort_values(by=0, ascending = False)

,team,event_type,0
73,Manchester City,PASS,795
119,Tottenham Hotspur,PASS,738
9,Arsenal,PASS,657
3,AFC Bournemouth,PASS,639
95,Southampton,PASS,562
30,Chelsea,PASS,552
44,Everton,PASS,539
80,Manchester United,PASS,511
66,Liverpool,PASS,499
125,Watford,PASS,427


In [24]:
eventdf.event_type.unique()

array(['CARD', 'FOUL_COMMITTED', 'GENERIC:generic', 'PASS', 'RECOVERY',
       'SHOT', 'TAKE_ON', 'BALL_OUT'], dtype=object)

In [25]:
eventdf[eventdf.groupby('event_type')[0].transform(max) == eventdf[0]]

,team,event_type,0
11,Arsenal,SHOT,27
28,Chelsea,CARD,6
43,Everton,GENERIC:generic,5
56,Leicester City,BALL_OUT,84
65,Liverpool,GENERIC:generic,5
67,Liverpool,RECOVERY,81
73,Manchester City,PASS,795
78,Manchester United,FOUL_COMMITTED,17
99,Stoke City,BALL_OUT,84
128,Watford,TAKE_ON,266


In [26]:
eventdf[eventdf[0] == eventdf.groupby('event_type')[0].transform('max')]

,team,event_type,0
11,Arsenal,SHOT,27
28,Chelsea,CARD,6
43,Everton,GENERIC:generic,5
56,Leicester City,BALL_OUT,84
65,Liverpool,GENERIC:generic,5
67,Liverpool,RECOVERY,81
73,Manchester City,PASS,795
78,Manchester United,FOUL_COMMITTED,17
99,Stoke City,BALL_OUT,84
128,Watford,TAKE_ON,266


In [27]:
teams = []
events = []
scores = []
score_type = []
for event in eventdf.event_type.unique():
    if event == 'GENERIC:generic':
        continue
    maxs = eventdf[eventdf[0] == eventdf[eventdf.event_type == event].max()[0]]
    mins = eventdf[eventdf[0] == eventdf[eventdf.event_type == event].min()[0]]
    for index, entry in maxs.iterrows():
        teams.append(entry.team)
        events.append(event)
        scores.append(entry[0])
        score_type.append('max')
    for index, entry in mins.iterrows():
        teams.append(entry.team)
        events.append(event)
        scores.append(entry[0])
        score_type.append('min')
maxminstats = {'teams': teams, 'events': events, 'scores': scores, 'score_type': score_type}
df = pd.DataFrame.from_dict(maxminstats)

In [28]:
eventdf.columns = ['team', 'event', 'score']
eventdf.to_csv('gameweek1.csv', index = False)

In [29]:
df[df.events == 'CARD']

,teams,events,scores,score_type
0,Brighton & Hove Albion,CARD,6,max
1,Brighton & Hove Albion,CARD,6,max
2,Chelsea,CARD,6,max
3,Crystal Palace,CARD,6,max
4,Newcastle United,CARD,6,max
5,AFC Bournemouth,CARD,1,min
6,Burnley,CARD,1,min
7,Crystal Palace,CARD,1,min
8,Everton,CARD,1,min
9,Leicester City,CARD,1,min


In [30]:
cards = eventdf[eventdf['event'] == 'CARD']

In [31]:
cards.to_csv("cards.csv", index = False)